# Import

In [ ]:
# some_file.py
import sys
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.append('/Users/matt/Documents/GitHub/qibo-cloud-backends_AWS/src/qibo_cloud_backends/')

import aws_client

# Latest aws_client.py

In [ ]:
from qibo.backends import NumpyBackend
from qibo.config import raise_error
from qibo.result import MeasurementOutcomes, QuantumState
from qibo import gates
from qibo import Circuit as QiboCircuit
from qibo.transpiler.pipeline import Passes, assert_transpiling
from qibo.transpiler.optimizer import Preprocessing
from qibo.transpiler.router import ShortestPaths
from qibo.transpiler.unroller import Unroller, NativeGates
from qibo.transpiler.placer import Random

import re
import importlib.util
import sys
import networkx as nx

from braket.aws import AwsDevice, AwsQuantumTask
from braket.circuits import Gate, observables
from braket.circuits import Circuit as BraketCircuit
from braket.devices import Devices, LocalSimulator

_QASM_BRAKET_GATES = {
    "id": "i",
    "cx": "cnot",
    "sx": "v",
    "sxdg": "vi",
    "sdg": "si",
    "tdg": "ti",
    "u3": "U",
}

class BraketClientBackend(NumpyBackend):
    """Backend for the remote execution of AWS circuits on the AWS backends.

    Args:
        device_arn (str | None): The ARN of the Braket device.
            If `None`, instantiates the `LocalSimulator("default")`.
    """

    def __init__(self, device=None, verbatim_circuit=False, transpilation=False, native_gates=None, coupling_map=None):
        """Initializes BraketBackend.

        Args:
            device (str): Default device is Braket's statevector LocalSimulator, LocalSimulator("default").
                Other devices are Braket's density matrix simulator, LocalSimulator("braket_dm"), or any other
                QPUs.
            verbatim_circuit (bool): If `True`, wrap the Braket circuit in a verbatim box to run it on the QPU
                without any transpilation. Defaults to `False`.
            transpilation (bool): If `True`, use Qibo's transpilation. Requires two additional arguments:
                native_gates and coupling_map.
            native_gates (list, qibo.gates): e.g. [gates.I, gates.RZ, gates.SX, gates.X, gates.ECR]
            coupling_map (list, list): E.g. [[0, 1], [0, 7], [1, 2], [2, 3], [4, 3], [4, 5], [6, 5], [7, 6]]
        """
        
        super().__init__()
        
        self.verbatim_circuit = verbatim_circuit

        self.transpilation = transpilation
        if transpilation:
            if coupling_map is None:
                raise_error(ValueError, "Expected qubit_map. E.g. qubit_map = [[0, 1], [0, 7], [1, 2], [2, 3], [4, 3], [4, 5], [6, 5], [7, 6]]")
            else:
                self.coupling_map = coupling_map
            if native_gates is None:
                raise_error(ValueError, "Expected native gates for transpilation. E.g. native_gates = [gates.I, gates.RZ, gates.SX, gates.X, gates.ECR]")
            else:
                self.native_gates = native_gates

        
        if device is None:
            self.device = LocalSimulator("default")
            # self.device = LocalSimulator("braket_dm")
        else:
            self.device = device
        self.name = "aws"

    def remove_qelib1_inc(self, qasm_string):
        """To remove the 'includes qe1lib.inc' from the OpenQASM string.

        Args: 
            qasm_code (OpenQASM circuit, str): circuit given in the OpenQASM format.

        Returns:
            qasm_code (OpenQASM circuit, str): circuit given in the OpenQASM format.
        """
        
        # Remove the "include "qelib1.inc";\n" line
        modified_code = re.sub(r'include\s+"qelib1.inc";\n', '', qasm_string)
        return modified_code
    
    def qasm_convert_gates(self, qasm_code):
        """To replace the notation for certain gates in OpenQASM

        Args: 
            qasm_code (OpenQASM circuit, str): circuit given in the OpenQASM format.

        Returns:
            qasm_code (OpenQASM circuit, str): circuit given in the OpenQASM format.
        """
        
        lines = qasm_code.split('\n')
        modified_code = ""
        for line in lines:
            for key in _QASM_BRAKET_GATES:
                if key in line:
                    line = line.replace(key, _QASM_BRAKET_GATES[key])
                    break
            modified_code += line + '\n'
        return modified_code

    def custom_connectivity(self, coupling_map):
        """Converts a coupling map given in list form to a networkx graph.
    
        Args:
            coupling_map (list): E.g. [[0, 1], [0, 7], [1, 2], [2, 3], [4, 3], [4, 5], [6, 5], [7, 6]]
        Returns:
            graph (networkx graph): graph
        """
        
        graph = nx.Graph()
        for connection in coupling_map:
            q1, q2 = connection
            graph.add_edge(q1, q2)
        return graph
    
    def transpile_qibo_to_qibo(self, circuit):
        """Transpiles a Qibo circuit with a specific topology specified by connectivity and custom_native_gates.
            There is no option for optimization_level like Qiskit. Therefore, no gates will be collapsed.
    
        Args:
            circuit (qibo.models.Circuit): Circuit to transpile
        Returns:
            transpiled_circuit (qibo.models.Circuit): Transpiled circuit.
            final_layout (dict): dict of connectivity?
        """
        
        # Define custom passes as a list
        custom_passes = []
        
        # Preprocessing adds qubits in the original circuit to match the number of qubits in the chip
        custom_passes.append(Preprocessing(connectivity=self.custom_connectivity(self.coupling_map)))
    
        # Placement step
        custom_passes.append(Random(connectivity=self.custom_connectivity(self.coupling_map))) 
    
        # Routing step
        custom_passes.append(ShortestPaths(connectivity=self.custom_connectivity(self.coupling_map)))
    
        # custom_native_gates = [gates.I, gates.RZ, gates.SX, gates.X, gates.ECR]
        custom_native_gates = [gates.I, gates.Z, gates.U3, gates.CZ]
        custom_passes.append(Unroller(native_gates=NativeGates.from_gatelist(self.native_gates))) # Gate decomposition ste
    
        custom_pipeline = Passes(custom_passes, 
                                 connectivity=self.custom_connectivity(self.coupling_map),
                                 native_gates=NativeGates.from_gatelist(self.native_gates)) 
                                # native_gates=NativeGates.default()
    
        transpiled_circuit, final_layout = custom_pipeline(circuit)
    
        return transpiled_circuit, final_layout

    def execute_circuit(self,
                        circuit,
                        nshots=1000,
                        **kwargs):
        """Executes a Qibo circuit on an AWS Braket device. The device defaults to the LocalSimulator().
            
        Args:
            circuit (qibo.models.Circuit): circuit to execute on the Braket device.
            nshots (int): Total number of shots.
        Returns:
            Measurement outcomes (qibo.measurement.MeasurementOutcomes): The outcome of the circuit execution.
        """
        
        measurements = circuit.measurements
        if not measurements:
            raise_error(RuntimeError, "No measurement found in the provided circuit.")
        nqubits = circuit.nqubits
        circuit_qasm = circuit.to_qasm()
        circuit_qasm = self.remove_qelib1_inc(circuit_qasm)
        circuit_qasm = self.qasm_convert_gates(circuit_qasm)
        braket_circuit = BraketCircuit.from_ir(circuit_qasm)

        if self.verbatim_circuit:
            braket_circuit = BraketCircuit().add_verbatim_box(braket_circuit)
        result = self.device.run(braket_circuit, shots=nshots).result()
        samples = result.measurements
        return MeasurementOutcomes(
            measurements=measurements, backend=self, samples=samples, nshots=nshots
        )


# 00. Extracting native gates from device

In [ ]:
from braket.aws import AwsDevice
from braket.devices import Devices, LocalSimulator

# device = LocalSimulator()
# device = 'arn:aws:braket:::device/quantum-simulator/amazon/sv1'
device = 'arn:aws:braket:us-east-1::device/qpu/ionq/Aria-1'

client = AwsDevice(device)
print(client.properties.paradigm.nativeGateSet)
print(client.properties)

# 0. Running a circuit on AWS

In [ ]:
from qibo import Circuit, gates
import numpy as np

circuit_qibo = Circuit(2)
circuit_qibo.add(gates.RX(0, np.pi/7))
circuit_qibo.add(gates.CNOT(1,0))
circuit_qibo.add(gates.M(0))
circuit_qibo.add(gates.M(1))
print('>> Original circuit')
print(circuit_qibo.draw())

# SV1 = "arn:aws:braket:::device/quantum-simulator/amazon/sv1"
# device = AwsDevice(SV1)
device = LocalSimulator("braket_dm")
AWS = BraketClientBackend(device = device)

out = AWS.execute_circuit(circuit_qibo, nshots=1000)
print(out.probabilities())

# 1. Instantiate a quantum circuit

In [ ]:
from qibo import Circuit, gates
import numpy as np
from qibo.backends import NumpyBackend
from qibo.models.error_mitigation import get_noisy_circuit, ZNE
from qibo.symbols import Z
from qibo.hamiltonians import SymbolicHamiltonian
from qibo.backends import GlobalBackend
from qibo.noise import DepolarizingError, NoiseModel, ReadoutError

circuit_qibo = Circuit(2)
circuit_qibo.add(gates.RX(0, np.pi/7))
circuit_qibo.add(gates.CNOT(1,0))
circuit_qibo.add(gates.M(0))
circuit_qibo.add(gates.M(1))
print('>> Original circuit')
print(circuit_qibo.draw())

# 2. noisy_circuit = get_noisy_circuit(...)

In [ ]:
noisy_circuit = get_noisy_circuit(circuit_qibo, num_insertions=5, insertion_gate = "CNOT")
print(noisy_circuit.draw())

# 3. Transpile. Use BraketClientBackend.transpile_qibo_to_qibo.
# Input native_gates, custom_coupling_map

In [ ]:
custom_coupling_map = [[0, 1]]#, [1, 2], [2, 3]] # [[0, 1], [0, 7], [1, 2], [2, 3], [4, 3], [4, 5], [6, 5], [7, 6]]
native_gates = [gates.I, gates.RZ, gates.U3, gates.CZ]

# SV1 = 'arn:aws:braket:::device/quantum-simulator/amazon/sv1'
# device = AwsDevice(SV1)
device = LocalSimulator("braket_dm")
AWS = BraketClientBackend(device = device, verbatim_circuit=False, transpilation=True, native_gates=native_gates, coupling_map=custom_coupling_map)

transpiled_circuit, _ = AWS.transpile_qibo_to_qibo(circuit_qibo)
print('>> Transpiled circuit')
print(transpiled_circuit.draw())


# 4. Set backend = BraketClientBackend

### a. Check that both original and transpiled circuit run properly on GlobalBackend()

In [ ]:
backend = GlobalBackend()

res_original = backend.execute_circuit(circuit_qibo, nshots=1000)
print(res_original.frequencies())

res_transpiled = backend.execute_circuit(transpiled_circuit, nshots=1000)
print(res_transpiled.frequencies())

# dir(backend)

### b. Now try AWS

In [ ]:
backend = AWS

res_original = backend.execute_circuit(circuit_qibo, nshots=1000)
print(res_original.frequencies())

res_transpiled = backend.execute_circuit(transpiled_circuit, nshots=1000)
print(res_transpiled.frequencies())


### Manually compute expectation value

In [ ]:
Zmatrix = gates.Z(0).matrix()
ZZmatrix = np.kron(Zmatrix, Zmatrix)
ZZdiag = np.real(np.diag(ZZmatrix))

dict = {}
for ii in range(2**circuit_qibo.nqubits):
    dict[bin(ii)[2:].zfill(circuit_qibo.nqubits)] = 0
for key, val in res_original.frequencies().items():
    dict[key] = val/sum(res_original.frequencies().values())

manual_expectation_val = 0
for key, val in dict.items():
    manual_expectation_val += val * int(ZZdiag[int(key, 2)])

print('Manually computing expectation value of ZZ operators = %.6f' %manual_expectation_val)


__Scenarios - failure:__

1. This circuit will fail because `q0` is not used. Transpiled circuit needs to be contiguous.
``` Python
q0: ──────────────
q1: ─U3─U3─Z─U3─M─
q2: ───────o─M──── 
```
Error message: `ValueError: Non-contiguous qubit indices supplied; qubit indices in a circuit must be contiguous.`

2. This circuit will fail because of some mismatch in dimensions.
``` Python
q0: ────────────────U3─U3─Z─U3─M─
q1: ─U3─Z─U3─o─U3─Z─U3────o─M────
q2: ────o─U3─Z─U3─o──────────────
q3: ─────────────────────────────
```
Error message: `ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 2 is different from 3)`

__Scenario - success:__

1. This scenario passes. The coupling map can't get any simpler: `custom_coupling_map = [[0, 1]]`
``` Python
q0: ─U3─U3─Z─U3─M─
q1: ───────o─M────
```

# 5. Run ZNE with backend = BraketClientBackend

In [ ]:
ZNE_circuit = transpiled_circuit # circuit_qibo
quantum_state = ZNE_circuit.execute().state()

obs = np.prod([Z(i) for i in range(ZNE_circuit.nqubits)])
print(obs)
obs_exact = SymbolicHamiltonian(obs, nqubits=ZNE_circuit.nqubits, backend=backend)
print(obs_exact)
obs = SymbolicHamiltonian(obs, backend=backend)
print(obs)

exact = obs_exact.expectation(quantum_state)
state = backend.execute_circuit(ZNE_circuit, nshots=10000)

print(state)

In [ ]:
estimate = ZNE(
    circuit=ZNE_circuit,
    observable=obs,
    noise_levels=np.array(range(5)),
    noise_model=None,
    nshots=100000,
    solve_for_gammas=False,
    insertion_gate="CNOT",
    readout=None,
    backend=backend,
)
print('ZNE estimated expectation value = %.6f' %estimate)

# Running on a Braket device?
### - Costs: DM1 & SV1: \\$0.075 per minute, TN1: \\$0.275 per minute
### - Run time: 